In [ ]:

#Import GPIO library 
import RPi.GPIO as GPIO 
import time
from picamera import PiCamera 
import picamera
from time import sleep 
import datetime 
import cv2
import numpy as np 
import math 

In [ ]:
relay_pins = 26
relay_pins1 = 13
led_pins = 21
led_pins1 = 2 
GPIO.setwarnings(False) 
GPIO.setmode(GPIO.BCM) 
GPIO.setup(relay_pins, GPIO.OUT) 
GPIO.setup(led_pins, GPIO.OUT) 
GPIO.output(relay_pins, 1) 
GPIO.setup(relay_pins1, GPIO.OUT) 
GPIO.output(relay_pins1, 1)
TRIG = 23 #Associate pin 23 to TRIG
ECHO = 24 #Associate pin 24 to ECHO 
print("Distance measurement in progress") 
GPIO.setup(TRIG,GPIO.OUT) #Set pin as GPIO out 
GPIO.setup(ECHO,GPIO.IN) #Set pin as GPIO in 


In [ ]:
while True:
    GPIO.output(TRIG, False) #Set TRIG as LOW 
    print("Waitng For Sensor To Settle")
    time.sleep(2) #Delay of 2 seconds 
    GPIO.output(TRIG, True) #Set TRIG as HIGH 
    time.sleep(0.00001) #Delay of 0.00001 seconds 
    GPIO.output(TRIG, False) #Set TRIG as LOW
    while GPIO.input(ECHO)==0: #Check whether the ECHO is LOW 
        pulse_start = time.time() #Saves the last known time of LOW pulse 
    while GPIO.input(ECHO)==1: #Check whether the ECHO is HIGH
        pulse_end = time.time() #Saves the last known time of HIGH pulse 
        pulse_duration = pulse_end - pulse_start #Get pulse duration to a variable 
        distance = pulse_duration * 17150
        distance = round(distance, 2) #Round to two decimal points 
        if distance > 2 and distance < 100:
            cap = cv2.VideoCapture(0) 
            while(cap.isOpened()):
                ret, img = cap.read() 
                cv2.rectangle(img,(500,500),(100,100),(0,255,0),0)
                crop_img = img[100:400, 100:400]
                grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY) 
                value = (35, 35)
                blurred = cv2.GaussianBlur(grey, value, 0)
                _, thresh1 = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                cv2.imshow('Thresholded', thresh1)
                contours, hierarchy = cv2.findContours(thresh1.copy(),cv2.RETR_TREE, \ cv2.CHAIN_APPROX_NONE)
                max_area = -1
                for i in range(len(contours)): 
                    cnt=contours[i]
                    area = cv2.contourArea(cnt) 
                    if(area>max_area):
                        max_area=area 
                        ci=i
                cnt=contours[ci]
                x,y,w,h = cv2.boundingRect(cnt) 
                cv2.rectangle(crop_img,(x,y),(x+w,y+h),(0,0,255),0)
                hull = cv2.convexHull(cnt)
                drawing = np.zeros(crop_img.shape,np.uint8) 
                cv2.drawContours(drawing,[cnt],0,(0,255,0),0) 
                cv2.drawContours(drawing,[hull],0,(0,0,255),0) 
                hull = cv2.convexHull(cnt,returnPoints = False)
                defects = cv2.convexityDefects(cnt,hull) 
                count_defects = 0
                cv2.drawContours(thresh1, contours, -1, (0,255,0), 3) 
                for i in range(defects.shape[0]):
                    s,e,f,d = defects[i,0] 
                    start = tuple(cnt[s][0]) 
                    end = tuple(cnt[e][0]) 
                    far = tuple(cnt[f][0])
                    a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2) 
                    angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
                    if angle <= 90: 
                        count_defects += 1
                        cv2.circle(crop_img,far,1,[0,0,255],-1) 
                    cv2.line(crop_img,start,end,[0,255,0],2)
                    if count_defects == 1:
                        cv2.putText(img,"Two", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                        GPIO.output(relay_pins, 0)
                        GPIO.output(relay_pins1, 1) 
                    if count_defects == 2:
                        str = "Three"
                        cv2.putText(img, str, (5,50), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
                        GPIO.output(relay_pins, 1)
                        GPIO.output(relay_pins1, 1) 
                    elif count_defects == 3:
                        cv2.putText(img,"Four", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                        GPIO.output(relay_pins1, 0)
                        GPIO.output(relay_pins, 1) 
                    elif count_defects == 4:
                        cv2.putText(img,"Five", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                        GPIO.output(relay_pins1, 1)
                        GPIO.output(relay_pins, 1) 
                    else:
                        cv2.putText(img,"", (50,50),cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
                        cv2.imshow('Gesture', img)
                        all_img = np.hstack((drawing, crop_img)) 
                        cv2.imshow('Contours', all_img)
                        k = cv2.waitKey(10) 
                        if k == 27:
                            break
        else:
            print("Out Of Range")